## Expert Knowledge Worker

A question answering agent that is an expert knowledge worker to be used by employees of Insurellm, an Insurance Tech company. The agent needs to be accurate and the solution should be low cost. This project will use <b>RAG (Retrieval Augmented Generation)</b> to ensure our question/answering assistant has high accuracy.

In [2]:
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [3]:
# imports for langchain
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [4]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"
db_name2 = "vector_db_nlp"

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [6]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

documents = []

loader = DirectoryLoader("NLP-lecture", glob="*.pdf", loader_cls=PyPDFLoader, silent_errors=True)
root_docs = loader.load()
for doc in root_docs:
    doc.metadata["doc_type"] = "root"   # or None
    documents.append(doc)


# Load PDFs inside sub-directories
subdirs = [p for p in glob.glob("NLP-lecture/*") if os.path.isdir(p)]
for folder in subdirs:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(
        folder,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
        silent_errors=True
    )
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [7]:
#split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [8]:
len(chunks)

844

In [9]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: lectures2025, root


## Chroma
We will be mapping each chunk of text into a Vector that represents the meaning of the text (embedding).
OpenAI offers a model to do this (Auto-Encoding LLM which generates an output given a complete input), which we will use by calling their API with some LangChain code.

In [10]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OpenAIEmbeddings()

# Alternative: Free Vector Embeddings from HuggingFace sentence-transformers
# Replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists
if os.path.exists(db_name2):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [11]:
# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name2)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 1688 documents


In [12]:
# Get one vector and find how many dimensions it has
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


## Visualizing the Vector Store

We look at the documents and their embedding vectors to see what's going on.

In [13]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'red'][['lectures2025', 'root'].index(t)] for t in doc_types]

In [14]:
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [15]:
# 3D visualization

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

# Use LangChain to bring it all together

In [28]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [29]:
query = "Can you tell me about the lectures in 2025? What are the topics covered in the first 5 weeks?"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

I don't know.


In [30]:
# Let's investigate what gets sent behind the scenes

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "Can you tell me about the lectures in 2025? What are the main topics covered?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Next lecture
Recurrent Neural Networks

Next lecture
Recurrent Neural Networks

Course Organization
• Scheduling of Q&A Session
• Last Exercise Sheet due today
2

Course Organization
• Scheduling of Q&A Session
• Last Exercise Sheet due today
2
Human: Can you tell me about the lectures in 2025? What are the main topics covered?

> Finished chain.

> Finished chain.

> Finished chain.

Answer: I'm sorry, but I don't know about the lectures in 2025 or the main topics covered.


In [31]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 1400})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [32]:
query = "Can you tell me about the lectures in 2025? What are the topics covered in the first 5 weeks?"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

In 2025, the lectures cover the following topics in the first 5 weeks:

- Week 1 (21 Oct 2025): Introduction to NLP
- Week 2 (28 Oct 2025): Text Preprocessing and Representation
- Week 3 (04 Nov 2025): Text Embeddings
- Week 4 (11 Nov 2025): Neural Networks (crash course)
- Week 5 (18 Nov 2025): Neural Models for NLP


In [33]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

##  Gradio using the Chat interface 

In [ ]:
# history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [35]:
# And in Gradio:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
